In [ ]:
import os
import re
import numpy as np
import pandas as pd
import cv2
import imageio
import xml.etree.ElementTree as ET

import czifile
import tifffile as tiff

from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local
from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball, binary_erosion
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops
from skimage.segmentation import watershed, find_boundaries
from scipy.ndimage import binary_fill_holes
import napari

# ===============================
# USER CONFIGURATION
# ===============================
# Path to your file (.tif or .czi)
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.tif"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/40A_UAS-TMEM192-3xHA x 40A 71G10 MARCM_around 12h - for quantification_3 Airy-CBs_300425.tif"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/TMEM-HA 6h CB airy_3_170722.tif"

# Correct microscope calibration (µm per pixel)
DEFAULT_VX_VY_UM = 0.04   # <-- TRUE XY SCALE you want to enforce
MAX_REASONABLE_VXY_UM = 0.5  # µm/px sanity limit

# ---------------------------
# Helper: refine radii by distance transform
# ---------------------------
def refine_radii_via_dt(img3d, blobs, win_px=25, bin_method="sauvola"):
    from skimage.filters import threshold_sauvola, threshold_local, threshold_otsu
    from skimage.morphology import remove_small_objects, disk, binary_opening
    from scipy.ndimage import distance_transform_edt as _edt
    from skimage.measure import label as _label

    if blobs is None or len(blobs) == 0:
        return blobs

    Z, H, W = img3d.shape
    out = blobs.copy().astype(np.float32)

    for i, (zc, yc, xc, _) in enumerate(out):
        z, y, x = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= z < Z and 0 <= y < H and 0 <= x < W):
            continue

        y1, y2 = max(0, y - win_px), min(H, y + win_px + 1)
        x1, x2 = max(0, x - win_px), min(W, x + win_px + 1)
        patch = img3d[z, y1:y2, x1:x2]

        if bin_method == "sauvola":
            ws = max(21, 2*(win_px//2)+1)
            thr = threshold_sauvola(patch, window_size=ws, k=0.2)
            bw = patch > thr
        elif bin_method == "local":
            ws = max(21, 2*(win_px//2)+1)
            thr = threshold_local(patch, block_size=ws, offset=-0.2*np.std(patch))
            bw = patch > thr
        else:
            try:
                thr = threshold_otsu(patch)
                bw = patch > thr
            except ValueError:
                continue

        bw = binary_opening(bw, footprint=disk(1))
        bw = remove_small_objects(bw, min_size=3, connectivity=2)

        yy, xx = y - y1, x - x1
        if not (0 <= yy < bw.shape[0] and 0 <= xx < bw.shape[1]) or not bw[yy, xx]:
            continue

        lab = _label(bw)
        lbl = lab[yy, xx]
        if lbl == 0:
            continue
        bw_obj = (lab == lbl)
        dt = _edt(bw_obj)
        r_px = float(dt[yy, xx])
        if r_px <= 0:
            continue
        out[i, 3] = r_px

    return out

# ===============================
# Image loading and metadata
# ===============================
def _try_float(x):
    try:
        return float(x)
    except:
        return None

def _parse_ome_xml(xml_text):
    if not xml_text:
        return None
    def grab(attr):
        m = re.search(fr'PhysicalSize{attr}="([\d\.eE+-]+)"', xml_text)
        return float(m.group(1)) if m else None
    return grab("X"), grab("Y"), grab("Z")

def load_any(file_path):
    ext = os.path.splitext(file_path)[1].lower()
    if ext in (".tif", ".tiff"):
        with tiff.TiffFile(file_path) as tf:
            arr = tf.asarray()
            try:
                ome_xml = tf.ome_metadata
            except Exception:
                ome_xml = None
            vx_um = vy_um = vz_um = None
            if ome_xml:
                vx_um, vy_um, vz_um = _parse_ome_xml(ome_xml)
        img = np.squeeze(arr)
        if img.ndim == 4:
            ch1, ch2 = (img[0], img[1]) if img.shape[0] == 2 else (img[:, 0], img[:, 1])
        else:
            raise RuntimeError("Unexpected TIFF shape")
        return ch1, ch2, (vx_um, vy_um, vz_um), {"type": "tiff"}
    else:
        raise ValueError("Unsupported file format")

# Load image and scales
img_ch1, img_ch2, (vx_um, vy_um, vz_um), meta = load_any(file_path)
if vx_um is None or vy_um is None:
    vx_um = vy_um = DEFAULT_VX_VY_UM
if vz_um is None:
    vz_um = vx_um

if vx_um > MAX_REASONABLE_VXY_UM:
    raise ValueError(f"XY pixel size too large: {vx_um} µm/px")

px_um_xy = float(np.sqrt(vx_um * vy_um))
voxel_um3 = vx_um * vy_um * vz_um
print(f"Voxel size (µm): X={vx_um}, Y={vy_um}, Z={vz_um}")

# ===== Aliases =====
image   = img_ch1       # lysosome channel
image_2 = img_ch2       # neuron / context channel

# ===============================
# Lysosome detection and scaling
# ===============================
image_smooth = gaussian(image, sigma=1.5)

blobs = blob_log(image_smooth, min_sigma=0.8, max_sigma=3.0, num_sigma=4, threshold=0.003)
if len(blobs) > 0:
    blobs[:, 3] *= np.sqrt(3)

blobs = refine_radii_via_dt(image_smooth, blobs)
if len(blobs) > 0:
    z_um = blobs[:, 0] * vz_um
    y_um = blobs[:, 1] * vy_um
    x_um = blobs[:, 2] * vx_um
    radius_um = blobs[:, 3] * px_um_xy
    diameter_um = 2 * radius_um
    volume_um3 = (4/3) * np.pi * radius_um**3
else:
    z_um = y_um = x_um = radius_um = diameter_um = volume_um3 = np.array([])

df = pd.DataFrame({
    "z_um": z_um,
    "y_um": y_um,
    "x_um": x_um,
    "radius_um": radius_um,
    "diameter_um": diameter_um,
    "volume_um3": volume_um3,
})
# add unique lysosome id for later tooltips
if len(df) > 0 and "id" not in df.columns:
    df["id"] = np.arange(1, len(df) + 1, dtype=int)

df.to_csv("lysosome_blobs_regions.csv", index=False)
print("Saved: lysosome_blobs_regions.csv")

# ==========================================
# 4) CH2: segmentation (CELL vs OUTSIDE)
# ==========================================
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
ch2 = gaussian(vol, sigma=1.5, preserve_range=True)

# Local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=301, offset=-0.2*np.std(R))
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=20000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(10))
neuron_mask = binary_erosion(neuron_mask, ball(7))

# Soma via distance + cleanup
dist = edt(neuron_mask)
cell_min_radius_vox = 1
cell_mask = (dist >= cell_min_radius_vox)
cell_mask &= neuron_mask
cell_mask = binary_opening(cell_mask, ball(5))
cell_mask = binary_closing(cell_mask, ball(10))
cell_mask = binary_fill_holes(cell_mask)

# Label and territories
body_lab = label(cell_mask, connectivity=3)
n_cells = int(body_lab.max())
print(f"Detected {n_cells} cells (soma).")

if n_cells > 0:
    dist_inside = edt(neuron_mask)
    cell_seg = watershed(-dist_inside, markers=body_lab, mask=neuron_mask)
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)

print("neuron voxels:", int(neuron_mask.sum()))
print("cell voxels:", int(cell_mask.sum()))

# ==========================================
# 5) Map lysosomes to (cell/outside) with per-cell IDs
# ==========================================
location_ch2 = []
cell_id_list = []

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue
        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0
            cell_id_list.append(cid)
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, lysosomes_with_cell_vs_outside.csv (µm-only metrics)")

# Include zero-count cells
try:
    if isinstance(cell_seg, np.ndarray) and cell_seg.max() > 0:
        all_cells = pd.DataFrame({
            "cell_id_ch2": np.arange(1, int(cell_seg.max()) + 1, dtype=int)
        })

        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts_nonzero = (
                df[df["location_ch2"] == "cell"]
                .groupby("cell_id_ch2")
                .size()
                .reset_index(name="count")
            )
        else:
            lys_counts_nonzero = pd.DataFrame(columns=["cell_id_ch2", "count"])

        lys_counts_all = (
            all_cells.merge(lys_counts_nonzero, on="cell_id_ch2", how="left")
                     .fillna({"count": 0})
        )
        lys_counts_all["count"] = lys_counts_all["count"].astype(int)

        lys_counts_all.to_csv("lysosome_counts_by_cell.csv", index=False)
        print("Updated: lysosome_counts_by_cell.csv now includes cells with 0 lysosomes.")
except Exception as e:
    print("Could not expand lysosome_counts_by_cell with zero-count cells:", e)

# ==========================================
# 6) Visualization (3 extra layers + tooltips)
# ==========================================
viewer = napari.Viewer()
viewer.add_image(img_ch2, name="Ch2 raw")
viewer.add_image(img_ch1, name="Ch1 raw")

# 1) each cell colored differently
cellid_layer = viewer.add_labels(
    cell_seg.astype(np.uint16),
    name='Cell ID (Ch2)',
    opacity=0.5
)
cellid_layer.blending = 'translucent_no_depth'

# boundaries (optional)
try:
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# 2) lysosomes inside vs outside as separate Points layers
def _mk_points_layer(coords_zyx, radii, name, face_color, props):
    sizes = np.clip(radii * 2, 2, None) if radii is not None else 3
    pts = viewer.add_points(
        coords_zyx,
        size=sizes,
        name=name
    )
    try:
        pts.face_color = face_color
        pts.edge_color = 'black'
        pts.edge_width = 0.3
        pts.properties = props
    except Exception:
        pass
    return pts

pts_in = pts_out = None
if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df and len(blobs) > 0:
    in_cell_mask = (df["location_ch2"].to_numpy() == "cell")
    out_cell_mask = ~in_cell_mask

    blobs_zyx = blobs[:, :3]
    radii_px  = blobs[:, 3] if blobs.shape[1] > 3 else np.ones(len(blobs))

    # inside
    if in_cell_mask.any():
        coords_in = blobs_zyx[in_cell_mask]
        radii_in = radii_px[in_cell_mask]
        props_in = {
            'lys_id':      df.loc[in_cell_mask, 'id'].to_numpy(),
            'cell':        df.loc[in_cell_mask, 'cell_id_ch2'].to_numpy(),
            'diameter_um': df.loc[in_cell_mask, 'diameter_um'].to_numpy()
        }
        pts_in = _mk_points_layer(coords_in, radii_in, 'Lysosomes (inside)', [0.0, 1.0, 1.0, 1.0], props_in)

    # outside
    if out_cell_mask.any():
        coords_out = blobs_zyx[out_cell_mask]
        radii_out = radii_px[out_cell_mask]
        props_out = {
            'lys_id':      df.loc[out_cell_mask, 'id'].to_numpy(),
            'cell':        np.zeros(out_cell_mask.sum(), dtype=int),
            'diameter_um': df.loc[out_cell_mask, 'diameter_um'].to_numpy()
        }
        pts_out = _mk_points_layer(coords_out, radii_out, 'Lysosomes (outside)', [1.0, 0.5, 0.0, 1.0], props_out)

# 3) Hover tooltip: cell id, lysosome id, diameter
def add_hover_tooltip(points_layer, units="µm"):
    if points_layer is None:
        return
    N = len(points_layer.data)
    points_layer.text = {
        'text': [''] * N,
        'size': 11,
        'color': 'yellow',
        'anchor': 'upper left'
    }

    def _on_move(layer, event):
        idx = layer.get_value(event.position, world=True)
        if idx is None or idx < 0 or idx >= len(layer.data):
            if isinstance(layer.text, dict) and isinstance(layer.text.get('text', None), list):
                if any(layer.text['text']):
                    layer.text = {**layer.text, 'text': [''] * len(layer.data)}
            return

        labels = [''] * len(layer.data)
        try:
            cid  = int(layer.properties['cell'][idx])
            lid  = int(layer.properties['lys_id'][idx])
            diam = float(layer.properties['diameter_um'][idx])
            labels[idx] = f"cell:{cid}  lys:{lid}  d:{diam:.2f} {units}"
        except Exception:
            pass

        layer.text = {
            'text': labels,
            'size': 11,
            'color': 'yellow',
            'anchor': 'upper left'
        }

    points_layer.mouse_move_callbacks.append(_on_move)

add_hover_tooltip(pts_in)
add_hover_tooltip(pts_out)

# ==========================================
# 7) Quick fused 2D video (optional)
# ==========================================
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)
    
    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                cv2.circle(overlay, (x, y), r, (255, 255, 0), 2)
    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# ==========================================
# 8) Per-cell rotation videos (X, Y, Z axes) – only for cells with lysosomes
# ==========================================
from scipy.ndimage import rotate as ndrotate

def _to_uint8(vol):
    v = vol.astype(np.float32)
    vmin, vmax = float(v.min()), float(v.max())
    if vmax <= vmin:
        return np.zeros_like(v, dtype=np.uint8)
    return (255.0 * (v - vmin) / (vmax - vmin)).astype(np.uint8)

def _rotate_points(pts_zyx, center_zyx, angle_deg, axis="Y"):
    if pts_zyx.size == 0:
        return pts_zyx
    theta = np.deg2rad(angle_deg)
    c, s = np.cos(theta), np.sin(theta)
    p = pts_zyx - center_zyx
    if axis.upper() == "Y":
        z = p[:, 0]*c + p[:, 2]*s
        y = p[:, 1]
        x = -p[:, 0]*s + p[:, 2]*c
    elif axis.upper() == "X":
        z = p[:, 0]
        y = p[:, 1]*c - p[:, 2]*s
        x = p[:, 1]*s + p[:, 2]*c
    else:
        z = p[:, 0]
        y = p[:, 1]*c + p[:, 2]*s
        x = -p[:, 1]*s + p[:, 2]*c
    return np.stack([z, y, x], axis=1) + center_zyx

def save_cell_spin_videos_allaxes(cell_labels, blobs, df,
                                  out_dir="cell_spin_videos",
                                  angles=tuple(range(0, 360, 10)),
                                  pad=8, fps=12,
                                  draw_point_radius_px=3):

    if not isinstance(cell_labels, np.ndarray) or cell_labels.max() == 0:
        print("No cells to export.")
        return
    if not isinstance(df, pd.DataFrame) or "cell_id_ch2" not in df.columns:
        print("Missing per-blob metadata; cannot filter by lysosome presence.")
        return

    cell_ids_with_lys = sorted(df.loc[
        (df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0),
        "cell_id_ch2"
    ].unique().astype(int))
    if len(cell_ids_with_lys) == 0:
        print("No cells contain lysosomes; no videos exported.")
        return

    os.makedirs(out_dir, exist_ok=True)
    COL_CYAN, COL_GREEN, TXT_COL = (255, 255, 0), (0, 255, 0), (240, 240, 240)
    props = [p for p in regionprops(cell_labels) if p.label in cell_ids_with_lys]
    axes_all = ["X", "Y", "Z"]
    Ztot, Ytot, Xtot = cell_labels.shape
    exported = 0

    for p in props:
        cid = p.label
        minz, miny, minx, maxz, maxy, maxx = p.bbox
        miny, maxy = max(0, miny - pad), min(Ytot, maxy + pad)
        minx, maxx = max(0, minx - pad), min(Xtot, maxx + pad)
        sub_lab = cell_labels[minz:maxz, miny:maxy, minx:maxx]
        if (sub_lab == cid).sum() == 0:
            continue
        subZ, subY, subX = sub_lab.shape
        center_zyx = np.array([(subZ-1)/2, (subY-1)/2, (subX-1)/2], dtype=np.float32)

        sub_blobs = df[(df["cell_id_ch2"] == cid) & (df["location_ch2"] == "cell")]
        if sub_blobs.empty:
            continue
        sel = sub_blobs.index.to_numpy()
        pts_local = np.stack([
            blobs[sel, 0]-minz,
            blobs[sel, 1]-miny,
            blobs[sel, 2]-minx
        ], axis=1).astype(np.float32)

        for axis in axes_all:
            frames = []
            for ang in angles:
                if axis == "Y":
                    rot_lab = ndrotate(sub_lab, angle=ang, axes=(0,2), reshape=False, order=0, mode='nearest')
                elif axis == "X":
                    rot_lab = ndrotate(sub_lab, angle=ang, axes=(1,0), reshape=False, order=0, mode='nearest')
                else:
                    rot_lab = ndrotate(sub_lab, angle=ang, axes=(1,2), reshape=False, order=0, mode='nearest')

                mask_rot = (rot_lab > 0)
                footprint = mask_rot.max(axis=0).astype(np.uint8)
                frame = np.zeros((subY, subX, 3), dtype=np.uint8)

                if footprint.any():
                    green = (footprint * 255)
                    overlay = frame.copy()
                    overlay[..., 1] = np.maximum(overlay[..., 1], green)
                    frame = cv2.addWeighted(frame, 1.0, overlay, 0.35, 0.0)
                    contours, _ = cv2.findContours((footprint*255), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    if contours:
                        cv2.drawContours(frame, contours, -1, COL_GREEN, 1, lineType=cv2.LINE_AA)

                pts_rot = _rotate_points(pts_local, center_zyx, ang, axis=axis)
                ys = np.clip(np.round(pts_rot[:, 1]).astype(int), 0, subY-1)
                xs = np.clip(np.round(pts_rot[:, 2]).astype(int), 0, subX-1)
                for k in range(len(xs)):
                    cv2.circle(frame, (int(xs[k]), int(ys[k])), draw_point_radius_px, COL_CYAN, -1, lineType=cv2.LINE_AA)

                frames.append(frame)

            out_path = os.path.join(out_dir, f"cell_{cid:03d}_spin{axis}.mp4")
            try:
                imageio.mimsave(out_path,
                                [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames],
                                fps=fps, format='FFMPEG')
                print(f"Saved: {out_path}")
            except TypeError:
                out_gif = out_path.replace(".mp4", ".gif")
                imageio.mimsave(out_gif,
                                [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames],
                                fps=fps)
                print(f"Saved (GIF): {out_gif}")
            exported += 1

    print(f"Exported {exported} per-cell rotation videos (X, Y, Z) → {out_dir}")

# ---- call it ----
save_cell_spin_videos_allaxes(cell_seg, blobs, df,
                              out_dir="cell_spin_videos",
                              angles=tuple(range(0, 360, 10)),
                              pad=8, fps=12)

# ==========================================
# 9) 3D screenshots (XY, YZ, XZ)
# ==========================================
def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (90, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 90, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_yz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XZ screenshot: {path}")

viewer.dims.ndisplay = 3
save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# ==========================================
# 10) EXTRA: Export Original stack (Ch1) as MP4
# ==========================================
try:
    raw_stack = np.array(image, dtype=np.float32)
    raw_norm = (255 * (raw_stack - raw_stack.min()) / (raw_stack.ptp() + 1e-8)).astype(np.uint8)

    frames_raw = []
    for z in range(raw_norm.shape[0]):
        frame_gray = raw_norm[z]
        frame_bgr = cv2.cvtColor(frame_gray, cv2.COLOR_GRAY2BGR)
        frames_raw.append(frame_bgr)

    mp4_name = "original_raw_ch1.mp4"
    imageio.mimsave(mp4_name, frames_raw, fps=8, format='FFMPEG')
    print(f"Saved: {mp4_name}")
except Exception as e:
    print("MP4 export of original stack failed:", e)

# ==========================================
# 11) EXTRA: Side-by-side (RAW | SEGMENTED) MP4
# ==========================================
try:
    def to_uint8_grayscale(vol):
        vol = vol.astype(np.float32)
        vmin, vmax = float(vol.min()), float(vol.max())
        if vmax <= vmin:
            return (np.zeros_like(vol, dtype=np.uint8))
        return (255.0 * (vol - vmin) / (vmax - vmin)).astype(np.uint8)

    raw_stack_u8 = to_uint8_grayscale(image)      # left panel (raw Ch1)
    seg_base_u8  = to_uint8_grayscale(image_2)    # right base (Ch2)

    Z, H, W = raw_stack_u8.shape
    fps = 8

    out_name = "raw_vs_segmented_side_by_side.mp4"
    writer = imageio.get_writer(out_name, fps=fps, format="FFMPEG")

    for z in range(Z):
        left_bgr = cv2.cvtColor(raw_stack_u8[z], cv2.COLOR_GRAY2BGR)
        base_bgr = cv2.cvtColor(seg_base_u8[z], cv2.COLOR_GRAY2BGR)

        if 'cell_mask' in globals():
            cell = (cell_mask[z].astype(np.uint8) * 255)
            overlay = base_bgr.copy()
            overlay[..., 1] = np.maximum(overlay[..., 1], cell)
            right_bgr = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
        else:
            right_bgr = base_bgr

        if 'blobs' in globals() and len(blobs) > 0 and 'cell_mask' in globals():
            z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
            for b in z_blobs:
                y, x = int(round(b[1])), int(round(b[2]))
                r = max(2, int(round(b[3])))
                if 0 <= y < H and 0 <= x < W and cell_mask[z, y, x]:
                    cv2.circle(right_bgr, (x, y), r, (255, 255, 0), 2)

        if left_bgr.shape != right_bgr.shape:
            right_bgr = cv2.resize(right_bgr, (left_bgr.shape[1], left_bgr.shape[0]),
                                   interpolation=cv2.INTER_NEAREST)
        divider = np.full((left_bgr.shape[0], 4, 3), 32, dtype=np.uint8)
        frame = cv2.hconcat([left_bgr, divider, right_bgr])

        writer.append_data(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    writer.close()
    print(f"Saved: {out_name}")

except Exception as e:
    try:
        print("FFMPEG writer failed; attempting GIF fallback. Error:", e)
        frames = []
        for z in range(raw_stack_u8.shape[0]):
            left_bgr = cv2.cvtColor(raw_stack_u8[z], cv2.COLOR_GRAY2BGR)
            base_bgr = cv2.cvtColor(seg_base_u8[z], cv2.COLOR_GRAY2BGR)
            if 'cell_mask' in globals():
                cell = (cell_mask[z].astype(np.uint8) * 255)
                overlay = base_bgr.copy()
                overlay[..., 1] = np.maximum(overlay[..., 1], cell)
                right_bgr = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
            else:
                right_bgr = base_bgr

            if 'blobs' in globals() and len(blobs) > 0 and 'cell_mask' in globals():
                z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
                for b in z_blobs:
                    y, x = int(round(b[1])), int(round(b[2]))
                    r = max(2, int(round(b[3])))
                    if 0 <= y < H and 0 <= x < W and cell_mask[z, y, x]:
                        cv2.circle(right_bgr, (x, y), r, (255, 255, 0), 2)

            if left_bgr.shape != right_bgr.shape:
                right_bgr = cv2.resize(right_bgr, (left_bgr.shape[1], left_bgr.shape[0]),
                                       interpolation=cv2.INTER_NEAREST)
            divider = np.full((left_bgr.shape[0], 4, 3), 32, dtype=np.uint8)
            frame = cv2.hconcat([left_bgr, divider, right_bgr])
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        imageio.mimsave("raw_vs_segmented_side_by_side.gif", frames, fps=8)
        print("Saved: raw_vs_segmented_side_by_side.gif")
    except Exception as e2:
        print("Side-by-side export failed:", e2)

# ==========================================
# 12) Run viewer
# ==========================================
napari.run()

C:\Users\nahue\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\nahue\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\nahue\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


Voxel size (µm): X=0.04, Y=0.04, Z=0.04
Saved: lysosome_blobs_regions.csv


In [ ]:
import os
import re
import numpy as np
import pandas as pd
import cv2
import imageio
import xml.etree.ElementTree as ET

import czifile
import tifffile as tiff

from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local, threshold_sauvola, threshold_otsu
from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball, binary_erosion, disk
)
from scipy.ndimage import distance_transform_edt as edt, binary_fill_holes
from skimage.measure import label, regionprops
from skimage.segmentation import watershed, find_boundaries
import napari

# ===============================
# USER CONFIGURATION
# ===============================
# Path to your file (.tif or .czi)
# file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.tif"
# file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/40A_UAS-TMEM192-3xHA x 40A 71G10 MARCM_around 12h - for quantification_3 Airy-CBs_300425.tif"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/TMEM-HA 6h CB airy_3_170722.tif"

# Correct microscope calibration (µm per pixel)
DEFAULT_VX_VY_UM = 0.04   # your true XY scale
MAX_REASONABLE_VXY_UM = 0.5  # µm/px sanity limit

# Physical bound for lysosome radius
MAX_RADIUS_UM = 0.4

# De-dup parameters
TOL_DUP_UM       = 1e-3     # radii within 0.001 µm considered identical
MAX_REL_DELTA    = 0.05     # ±5%
FALLBACK_EPS_UM  = 1e-4     # tiny tie-breaker for remaining ties
WIN_PX           = 25       # local window half-size for re-estimation
SMOOTH_SIGMA     = 1.5      # same smoothing as detection
MIN_OBJ_PX       = 3        # remove tiny particles inside local patch
BIN_METHOD       = "sauvola"  # "sauvola" | "local" | "otsu"

# ==========================================
# Helper: refine radii by distance transform
# ==========================================
def refine_radii_via_dt(img3d, blobs, win_px=25, bin_method="sauvola"):
    if blobs is None or len(blobs) == 0:
        return blobs

    Z, H, W = img3d.shape
    out = blobs.copy().astype(np.float32)

    for i, (zc, yc, xc, _) in enumerate(out):
        z, y, x = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= z < Z and 0 <= y < H and 0 <= x < W):
            continue

        y1, y2 = max(0, y - win_px), min(H, y + win_px + 1)
        x1, x2 = max(0, x - win_px), min(W, x + win_px + 1)
        patch = img3d[z, y1:y2, x1:x2]

        if bin_method == "sauvola":
            ws = max(21, 2*(win_px//2)+1)
            thr = threshold_sauvola(patch, window_size=ws, k=0.2)
            bw = patch > thr
        elif bin_method == "local":
            ws = max(21, 2*(win_px//2)+1)
            thr = threshold_local(patch, block_size=ws, offset=-0.2*np.std(patch))
            bw = patch > thr
        else:
            try:
                thr = threshold_otsu(patch)
                bw = patch > thr
            except ValueError:
                continue

        bw = binary_opening(bw, footprint=disk(1))
        bw = remove_small_objects(bw, min_size=MIN_OBJ_PX, connectivity=2)

        yy, xx = y - y1, x - x1
        if not (0 <= yy < bw.shape[0] and 0 <= xx < bw.shape[1]) or not bw[yy, xx]:
            continue

        lab = label(bw)
        lbl = lab[yy, xx]
        if lbl == 0:
            continue
        bw_obj = (lab == lbl)
        dt = edt(bw_obj)
        r_px = float(dt[yy, xx])
        if r_px <= 0:
            continue
        out[i, 3] = r_px

    return out

# ===============================
# Image loading and metadata
# ===============================
def _parse_ome_xml(xml_text):
    if not xml_text:
        return None, None, None
    def grab(attr):
        m = re.search(fr'PhysicalSize{attr}="([\d\.eE+-]+)"', xml_text)
        return float(m.group(1)) if m else None
    return grab("X"), grab("Y"), grab("Z")

def load_any(file_path):
    ext = os.path.splitext(file_path)[1].lower()
    if ext in (".tif", ".tiff"):
        with tiff.TiffFile(file_path) as tf:
            arr = tf.asarray()
            try:
                ome_xml = tf.ome_metadata
            except Exception:
                ome_xml = None
            vx_um = vy_um = vz_um = None
            if ome_xml:
                vx_um, vy_um, vz_um = _parse_ome_xml(ome_xml)
        img = np.squeeze(arr)
        if img.ndim == 4:
            ch1, ch2 = (img[0], img[1]) if img.shape[0] == 2 else (img[:, 0], img[:, 1])
        else:
            raise RuntimeError("Unexpected TIFF shape")
        return ch1, ch2, (vx_um, vy_um, vz_um), {"type": "tiff"}
    else:
        raise ValueError("Unsupported file format")

# Load image and scales
img_ch1, img_ch2, (vx_um, vy_um, vz_um), meta = load_any(file_path)
if vx_um is None or vy_um is None:
    vx_um = vy_um = DEFAULT_VX_VY_UM
if vz_um is None:
    vz_um = vx_um

if vx_um > MAX_REASONABLE_VXY_UM:
    raise ValueError(f"XY pixel size too large: {vx_um} µm/px")

px_um_xy = float(np.sqrt(vx_um * vy_um))
voxel_um3 = vx_um * vy_um * vz_um
print(f"Voxel size (µm): X={vx_um}, Y={vy_um}, Z={vz_um}")

# ===== Aliases used throughout =====
image   = img_ch1       # lysosome channel
image_2 = img_ch2       # neuron / context channel

# ===============================
# 1) Lysosome detection and scaling
# ===============================
image_smooth = gaussian(image, sigma=1.5)

blobs = blob_log(image_smooth, min_sigma=0.8, max_sigma=3.0, num_sigma=4, threshold=0.003)
if len(blobs) > 0:
    blobs[:, 3] *= np.sqrt(3)

blobs = refine_radii_via_dt(image_smooth, blobs)
if len(blobs) > 0:
    z_um = blobs[:, 0] * vz_um
    y_um = blobs[:, 1] * vy_um
    x_um = blobs[:, 2] * vx_um
    radius_um = blobs[:, 3] * px_um_xy
    diameter_um = 2 * radius_um
    volume_um3 = (4/3) * np.pi * radius_um**3
else:
    z_um = y_um = x_um = radius_um = diameter_um = volume_um3 = np.array([])

df = pd.DataFrame({
    "z_um": z_um,
    "y_um": y_um,
    "x_um": x_um,
    "radius_um": radius_um,
    "diameter_um": diameter_um,
    "volume_um3": volume_um3,
})
# add stable integer IDs if missing (used by tooltips)
if "id" not in df.columns:
    df["id"] = np.arange(1, len(df) + 1, dtype=int)

df.to_csv("lysosome_blobs_regions.csv", index=False)
print("Saved: lysosome_blobs_regions.csv")

# ===============================
# 2) De-duplicate radius_um (±5%, bound to [0, 0.4] µm)
# ===============================
def find_duplicate_groups(values_um, tol_um=TOL_DUP_UM):
    vals = np.asarray(values_um, dtype=float)
    if vals.size == 0:
        return []
    order = np.argsort(vals)
    groups, cur = [], [int(order[0])]
    for a, b in zip(order[:-1], order[1:]):
        if abs(vals[b] - vals[a]) <= tol_um:
            if cur and cur[-1] != a:
                cur.append(int(a))
            cur.append(int(b))
        else:
            if len(set(cur)) > 1:
                groups.append(sorted(set(cur)))
            cur = [int(b)]
    if len(set(cur)) > 1:
        groups.append(sorted(set(cur)))
    # unique groups
    final, seen = [], set()
    for g in groups:
        t = tuple(g)
        if t not in seen:
            final.append(g)
            seen.add(t)
    return final

def binarize_patch(patch, method="sauvola"):
    if method == "sauvola":
        ws = max(21, 2*(WIN_PX//2)+1)
        thr = threshold_sauvola(patch, window_size=ws, k=0.2)
        bw = patch > thr
    elif method == "local":
        ws = max(21, 2*(WIN_PX//2)+1)
        thr = threshold_local(patch, block_size=ws, offset=-0.2*np.std(patch))
        bw = patch > thr
    else:
        try:
            thr = threshold_otsu(patch)
            bw = patch > thr
        except ValueError:
            return None
    bw = binary_opening(bw, footprint=disk(1))
    bw = remove_small_objects(bw, min_size=MIN_OBJ_PX, connectivity=2)
    return bw

def reestimate_radius_um(img3d, zyx_um, scales_um=(vz_um, vy_um, vx_um)):
    vz, vy, vx = scales_um
    zc_um, yc_um, xc_um = zyx_um
    zc = int(round(zc_um / vz))
    yc = int(round(yc_um / vy))
    xc = int(round(xc_um / vx))
    Z, H, W = img3d.shape
    if not (0 <= zc < Z and 0 <= yc < H and 0 <= xc < W):
        return None
    y1, y2 = max(0, yc - WIN_PX), min(H, yc + WIN_PX + 1)
    x1, x2 = max(0, xc - WIN_PX), min(W, xc + WIN_PX + 1)
    if y2 - y1 < 3 or x2 - x1 < 3:
        return None
    patch = img3d[zc, y1:y2, x1:x2].astype(np.float32)
    patch = gaussian(patch, sigma=SMOOTH_SIGMA, preserve_range=True)
    bw = binarize_patch(patch, BIN_METHOD)
    if bw is None or not bw.any():
        return None
    yy, xx = yc - y1, xc - x1
    if not (0 <= yy < bw.shape[0] and 0 <= xx < bw.shape[1]) or not bw[yy, xx]:
        return None
    lab = label(bw)
    lbl = lab[yy, xx]
    if lbl == 0:
        return None
    bw_obj = (lab == lbl)
    dt = edt(bw_obj).astype(np.float32)
    r_px = float(dt[yy, xx])
    if r_px <= 0:
        return None
    r_um = r_px * np.sqrt(vx * vy)
    return r_um

def clamp_within_bounds_and_relative(original, candidate, max_radius, rel_delta):
    phys_lo, phys_hi = 0.0, max_radius
    rel_lo,  rel_hi  = original * (1.0 - rel_delta), original * (1.0 + rel_delta)
    # handle near-zero original gracefully
    lo = max(phys_lo, min(rel_lo, rel_hi))
    hi = min(phys_hi, max(rel_lo, rel_hi))
    if lo > hi:
        # choose feasible value closest to original
        if original < phys_lo:
            return phys_lo
        if original > phys_hi:
            return phys_hi
        return np.clip(original, phys_lo, phys_hi)
    return float(np.clip(candidate, lo, hi))

# keep a copy of the original radii for strict ±5% limits during tie-breaking
orig_radius_arr = df["radius_um"].to_numpy(dtype=float)

rad = df["radius_um"].to_numpy(dtype=float)
dup_groups = find_duplicate_groups(rad, TOL_DUP_UM)
over_idx = np.where(rad > MAX_RADIUS_UM)[0].tolist()

to_fix = sorted(set([i for g in dup_groups for i in g] + over_idx))
report_rows = []

if to_fix:
    print(f"Re-estimating {len(to_fix)} radii from raw image (±5% bound + [0,0.4] µm)…")
    for i in to_fix:
        old = float(df.at[i, "radius_um"])
        zyx_um = (float(df.at[i, "z_um"]), float(df.at[i, "y_um"]), float(df.at[i, "x_um"]))
        new_r = reestimate_radius_um(image, zyx_um, (vz_um, vy_um, vx_um))
        reason = []
        if i in over_idx: reason.append(">0.4")
        if any(i in g for g in dup_groups): reason.append("duplicate")
        if new_r is None:
            cand = old
        else:
            cand = new_r
        new_r2 = clamp_within_bounds_and_relative(old, cand, MAX_RADIUS_UM, MAX_REL_DELTA)
        if abs(new_r2 - old) > 1e-12:
            df.at[i, "radius_um"]   = new_r2
            df.at[i, "diameter_um"] = 2.0 * new_r2
            df.at[i, "volume_um3"]  = (4.0/3.0) * np.pi * (new_r2**3)
        report_rows.append([int(df.at[i, "id"]), old, new_r2, "|".join(reason) if reason else "adjusted"])

    # second pass: tiny epsilon offsets if any ties remain (still within each entry’s ±5% of ORIGINAL)
    rad2 = df["radius_um"].to_numpy(dtype=float)
    remaining = find_duplicate_groups(rad2, TOL_DUP_UM)
    for g in remaining:
        sub = df.loc[g, ["z_um","y_um","x_um","id","radius_um"]].copy()
        order = np.lexsort((sub["id"].to_numpy(),
                            sub["x_um"].to_numpy(),
                            sub["y_um"].to_numpy(),
                            sub["z_um"].to_numpy()))
        idxs = np.array(g)[order]
        for rank, idx in enumerate(idxs):
            if rank == 0:
                continue
            current = float(df.at[idx, "radius_um"])
            orig   = float(orig_radius_arr[idx])  # strict ±5% guard uses ORIGINAL pre-postprocess value
            eps_signed = (rank - (len(idxs)-1)/2.0) * FALLBACK_EPS_UM
            cand = current + eps_signed
            cand2 = clamp_within_bounds_and_relative(orig, cand, MAX_RADIUS_UM, MAX_REL_DELTA)
            if abs(cand2 - current) > 1e-12:
                df.at[idx, "radius_um"]   = cand2
                df.at[idx, "diameter_um"] = 2.0 * cand2
                df.at[idx, "volume_um3"]  = (4.0/3.0) * np.pi * (cand2**3)
                report_rows.append([int(df.at[idx,"id"]), current, cand2, "epsilon_tie_break"])

    df.to_csv("lysosome_blobs_regions_dedup.csv", index=False)
    pd.DataFrame(report_rows, columns=["id","old_radius_um","new_radius_um","reason"]).to_csv(
        "lysosome_radius_dedup_report.csv", index=False
    )
    print("Saved: lysosome_blobs_regions_dedup.csv, lysosome_radius_dedup_report.csv")
else:
    print("No duplicate/out-of-range radii detected; skipping de-dup.")
    df.to_csv("lysosome_blobs_regions_dedup.csv", index=False)

# Also keep df in-memory updated so downstream layers/tooltips use deduped values
# (comment the next line if you prefer using the raw values in the viewer)
# df = pd.read_csv("lysosome_blobs_regions_dedup.csv")

# ===============================
# 3) CH2: segmentation (CELL vs OUTSIDE)
# ===============================
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
ch2 = gaussian(vol, sigma=1.5, preserve_range=True)

# Local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=301, offset=-0.2*np.std(R))
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=20000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(10))
neuron_mask = binary_erosion(neuron_mask, ball(7))

# Soma via distance + cleanup
dist = edt(neuron_mask)
cell_min_radius_vox = 1
cell_mask = (dist >= cell_min_radius_vox)
cell_mask &= neuron_mask
cell_mask = binary_opening(cell_mask, ball(5))
cell_mask = binary_closing(cell_mask, ball(10))
cell_mask = binary_fill_holes(cell_mask)

# Label and territories
body_lab = label(cell_mask, connectivity=3)
n_cells = int(body_lab.max())
print(f"Detected {n_cells} cells (soma).")

if n_cells > 0:
    dist_inside = edt(neuron_mask)
    cell_seg = watershed(-dist_inside, markers=body_lab, mask=neuron_mask)
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)

print("neuron voxels:", int(neuron_mask.sum()))
print("cell voxels:", int(cell_mask.sum()))

# ==========================================
# 4) Map lysosomes to (cell/outside) with per-cell IDs
# ==========================================
location_ch2 = []
cell_id_list = []

if len(df) > 0:
    Z, Y, X = neuron_mask.shape
    # reconstruct voxel indices from µm coords (center)
    z_idx = np.clip(np.round(df["z_um"].to_numpy() / vz_um).astype(int), 0, Z-1) if Z>0 else np.array([])
    y_idx = np.clip(np.round(df["y_um"].to_numpy() / vy_um).astype(int), 0, Y-1) if Y>0 else np.array([])
    x_idx = np.clip(np.round(df["x_um"].to_numpy() / vx_um).astype(int), 0, X-1) if X>0 else np.array([])

    for zz, yy, xx in zip(z_idx, y_idx, x_idx):
        if Z==0:
            location_ch2.append("outside"); cell_id_list.append(0); continue
        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0
            cell_id_list.append(cid)
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, lysosomes_with_cell_vs_outside.csv")

# ==========================================
# 5) Napari visualization (requested layers)
# ==========================================
viewer = napari.Viewer()
viewer.add_image(img_ch2, name="Ch2 raw")
viewer.add_image(img_ch1, name="Ch1 raw")

# (a) Each cell with a different color
cellid_layer = viewer.add_labels(
    cell_seg.astype(np.uint16),
    name='Cell ID (Ch2)',
    opacity=0.5
)
cellid_layer.blending = 'translucent_no_depth'
boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
viewer.add_image(
    boundaries.astype(np.uint8),
    name='Cell ID boundaries',
    blending='additive',
    contrast_limits=(0, 1),
    colormap='magenta',
    opacity=0.6
)

# (b) Lysosomes inside vs outside (two separate point layers)
if len(df) > 0:
    in_mask  = (df["location_ch2"].to_numpy() == "cell")
    out_mask = ~in_mask

    # points positions from µm back to voxel indices
    blobs_vox = np.stack([
        np.round(df["z_um"].to_numpy() / vz_um),
        np.round(df["y_um"].to_numpy() / vy_um),
        np.round(df["x_um"].to_numpy() / vx_um)
    ], axis=1).astype(float)

    # sizes from radius_um (voxel units)
    radii_vox = (df["radius_um"].to_numpy() / px_um_xy).astype(float)
    sizes = np.clip(radii_vox * 2.0, 2.0, None)

    if in_mask.any():
        pts_in = viewer.add_points(
            blobs_vox[in_mask, :],
            size=sizes[in_mask],
            name='Lysosomes (inside)',
            face_color=[0.0, 1.0, 1.0, 1.0],  # cyan
            edge_color='black',
            edge_width=0.3
        )
        # hover labels: cell id, lysosome id, diameter
        pts_in.properties = {
            'cell_id':    df.loc[in_mask, 'cell_id_ch2'].to_numpy(),
            'lys_id':     df.loc[in_mask, 'id'].to_numpy(),
            'diameter_um':df.loc[in_mask, 'diameter_um'].to_numpy()
        }
        pts_in.text = {'text': 'C:{cell_id}  ID:{lys_id}  d:{diameter_um:.3f}µm',
                       'size': 10, 'color': 'yellow', 'anchor': 'upper left'}

    if out_mask.any():
        pts_out = viewer.add_points(
            blobs_vox[out_mask, :],
            size=sizes[out_mask],
            name='Lysosomes (outside)',
            face_color=[1.0, 0.5, 0.0, 1.0],  # orange
            edge_color='black',
            edge_width=0.3
        )
        pts_out.properties = {
            'cell_id':    df.loc[out_mask, 'cell_id_ch2'].to_numpy(),
            'lys_id':     df.loc[out_mask, 'id'].to_numpy(),
            'diameter_um':df.loc[out_mask, 'diameter_um'].to_numpy()
        }
        pts_out.text = {'text': 'C:{cell_id}  ID:{lys_id}  d:{diameter_um:.3f}µm',
                        'size': 10, 'color': 'white', 'anchor': 'upper left'}

# (c) Optional: a semi-transparent single mask layer for “cell vs outside”
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (mask)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

# ==========================================
# 6) Quick fused 2D video (inside only overlays)
# ==========================================
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    # draw only in-cell points on this slice
    if len(df) > 0:
        z_idx = np.round(df["z_um"].to_numpy() / vz_um).astype(int)
        this = np.where((df["location_ch2"].to_numpy() == "cell") & (z_idx == z))[0]
        for k in this:
            y = int(np.round(df.at[k, "y_um"] / vy_um))
            x = int(np.round(df.at[k, "x_um"] / vx_um))
            r_vox = int(max(2, np.round(df.at[k, "radius_um"] / px_um_xy)))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                cv2.circle(overlay, (x, y), r_vox, (255, 255, 0), 2)  # yellow

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# ==========================================
# 7) Run viewer
# ==========================================
napari.run()